Key Difference Between MLE and MAP is the regression coefficient like θ or W in this case

For MLE, W is fixed. but in MAP W is a random variable.  
For MLE we don't need assumption besides prediction model. But in MAP we need "assume" a "prior" to maximize the posterior  
Which result in MAP being more "subjective".  
With good prior, we can fit target more well than MLE  
, but with bad prior, we could lead to a result far from correct answer
  
Overall, MAP can be seen as a regulized approch similar to MLE


**------------------------------------Initializing--------------------------------------------**

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
wine_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/X.csv")
wine_target=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/T.csv")

wine_features=["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]
X=wine_data[wine_features]
Y=wine_target

**------------------------------------Func used in last part--------------------------------------------**

In [3]:
def split_DataFrame(X,N): #Can't handle Series
    df_1=X.iloc[:N,:]
    df_2=X.iloc[N:,:]
    return df_1, df_2
def variance_numpy(W0,W,X,Y):
    N,D=X.shape
    count=0
    for i in range(N):
        count+=(predict(W0, W, X[i])-Y[i])**2
    count/=N
    return count
def gaussian(x,u,var):
    deviation=np.sqrt(var)
    tmp=np.sqrt(2*np.pi)
    probability=np.exp(-(x-u)**2/(2*var))/(deviation*tmp)
    return probability
def predict(W0,W1,X):
    X=X.reshape(len(X),1)
    return np.matmul(W1,X)+W0
def regression_M(X,Y,M):
    X_numpy=X.to_numpy()
    Y_numpy=Y.to_numpy()
    H=X_numpy
    N, D=X_numpy.shape
    i=0
    for m in range(1,M):
        coefs=H.size/N
        while i<coefs :
            for j in range(D):
                tmp=H[:,i]*X_numpy[:,j]
                tmp=tmp.reshape(N,1)
                H=np.append(H,tmp,1)
                #print(H.shape)
            i+=1
    
    ones=np.array([[1]]*N)  
    H=np.append(H,ones,1)   #for intercept
    H_trans=np.transpose(H)
#-------Normal Eqution-----
    mul=np.matmul(H_trans,H)
    tmp=np.linalg.pinv(mul)
    W=np.matmul(np.matmul(tmp,H_trans),Y_numpy)

    intercept=W[-1]
    W=np.delete(W,-1)
    H=np.delete(H,-1,1)
    return intercept, W, H
def N_set(data,cut_size):
  data_set=[]
  N,D=data.shape
  cur=0
  while cur<(N-cut_size):
    data_set.append(data.iloc[cur:cur+cut_size,:])
    cur+=cut_size
    
  data_set.append(data.iloc[cur:,:])
  return data_set

**------------------------------------New Func Define--------------------------------------------**

In [4]:
def vector_phi(x,M):
    D=len(x)
    phi=np.ones(1)
    cur=0
    for i in range(M):
        last_len=len(phi)
        while cur<last_len:
            for j in range(D):
                phi=np.append(phi,phi[cur]*x[j])
            cur+=1
    phi=np.reshape(phi,(len(phi),1))
    return phi
def Matrix_S(x_in,X,a,B,M):
    N,D=X.shape

    phi_in=vector_phi(x_in,M)   
    D_phi=len(phi_in)
    phi_in_t=np.transpose(phi_in)
    I=np.identity(D_phi)
    summation=np.zeros((D_phi,D_phi))

    for i in range(N):
        phi=vector_phi(X[i],M)
        phi_t=np.transpose(phi)
        summation+=phi@phi_t

    S_inv=a*I+B*summation
    S=np.linalg.pinv(S_inv)
    return S
def mean_MAP(x_in, X, T, a, B, M, S):
    N,D=X.shape
    phi_in=vector_phi(x_in,M)
    phi_in_t=np.transpose(phi_in)
    D_phi=len(phi_in)
    
    summation=np.zeros((D_phi,1))
    for i in range(N):
        summation+=(T[i]*vector_phi(X[i],M))
    
    mean=B*phi_in_t@S@summation # @=>matmul
    return mean
def var_MAP(x_in,B,M,S):
    phi_in=vector_phi(x_in,M)
    phi_in_t=np.transpose(phi_in)
    return (1/B)+phi_in_t@S@phi_in

**------------------------------------Below is the code for 2.3(b)--------------------------------------------**

In [5]:
#--------------HyperParameter--------------------#
a=0.001
#B=0.2  #use BML in below
#order=1 #below will be define
max_order=2 #not doing M=3 it cost too much time
#------------------------------------------------#

X_train, X_test=split_DataFrame(X,1500)
Y_train, Y_test=split_DataFrame(Y,1500)
X_train_numpy=X_train.to_numpy()
Y_train_numpy=Y_train.to_numpy()
X_test_numpy=X_test.to_numpy()
Y_test_numpy=Y_test.to_numpy()


for order in range(1,max_order+1):
  #--------use B ML-------------#
  W0, W, H_train=regression_M(X_train,Y_train,order)
  var_reg=variance_numpy(W0, W, H_train, Y_train_numpy)
  B=1/var_reg
  #-------------------------------#
  log_likelihood=np.float64(0)
  N,D=X_test.shape

  for i in range(N):
      S=Matrix_S(X_test_numpy[i],X_train_numpy,a,B,order)
      mean=mean_MAP(X_test_numpy[i], X_train_numpy, Y_train_numpy,a, B, order, S)
      var=var_MAP(X_test_numpy[i], B, order, S)
      p=gaussian(Y_test_numpy[i],mean,var)
      log_likelihood+=np.log(p)
  print("M=",order, "avg log_likelihood of test",log_likelihood/N)

M= 1 avg log_likelihood of test [[-0.8956122]]
M= 2 avg log_likelihood of test [[-0.97806948]]


Cross Validation

In [9]:
def Validation_MAP_poly(X_train,X_test,Y_train,Y_test,a,max_order,if_print=True):
  X_train_numpy=X_train.to_numpy()
  Y_train_numpy=Y_train.to_numpy()
  X_test_numpy=X_test.to_numpy()
  Y_test_numpy=Y_test.to_numpy()
  ret=[]

  for order in range(1,max_order+1):
    #--------use B ML-------------#
    W0, W, H_train=regression_M(X_train,Y_train,order)
    var_reg=variance_numpy(W0, W, H_train, Y_train_numpy)
    B=1/var_reg
    #-------------------------------#
    log_likelihood=np.float64(0)
    N,D=X_test.shape

    for i in range(N):
      S=Matrix_S(X_test_numpy[i],X_train_numpy,a,B,order)
      mean=mean_MAP(X_test_numpy[i], X_train_numpy, Y_train_numpy,a, B, order, S)
      var=var_MAP(X_test_numpy[i], B, order, S)
      p=gaussian(Y_test_numpy[i],mean,var)
      log_likelihood+=np.log(p)
      
    log_likelihood/=N  
    if if_print:
      print("M=",order, "avg log_likelihood of test",log_likelihood)
    ret.append(log_likelihood)
 
  return ret

In [10]:
cut_size=100
Max_poly_order=1
a=0 #perform well in former example

X_sets=N_set(X,cut_size)
Y_sets=N_set(Y,cut_size)
times=len(X_sets)

count=[np.float64(0)]*Max_poly_order
for t in range(times):
  print("----------------time=",t,"-------------------------")
  X_train=pd.DataFrame()
  Y_train=pd.DataFrame()
  X_test=X_sets[t].copy()
  Y_test=Y_sets[t].copy()
  for i in range(times):
    if i==t :
      continue
    else:
      X_train=X_train.append(X_sets[i])
      Y_train=Y_train.append(Y_sets[i])
  tmp=Validation_MAP_poly(X_train, X_test, Y_train, Y_test,a, Max_poly_order)
  for i in range(Max_poly_order):
    count[i]+=tmp[i]
print("############################################")
for i in range(Max_poly_order):
  print("average likelihood of M=",i+1,": ",count[i]/times)

----------------time= 0 -------------------------
M= 1 avg log_likelihood of test [[-1.06815721]]
----------------time= 1 -------------------------
M= 1 avg log_likelihood of test [[-0.87325257]]
----------------time= 2 -------------------------
M= 1 avg log_likelihood of test [[-1.02279663]]
----------------time= 3 -------------------------
M= 1 avg log_likelihood of test [[-0.98222199]]
----------------time= 4 -------------------------
M= 1 avg log_likelihood of test [[-1.15473474]]
----------------time= 5 -------------------------
M= 1 avg log_likelihood of test [[-0.95008983]]
----------------time= 6 -------------------------
M= 1 avg log_likelihood of test [[-1.00592418]]
----------------time= 7 -------------------------
M= 1 avg log_likelihood of test [[-0.86043515]]
----------------time= 8 -------------------------
M= 1 avg log_likelihood of test [[-1.10858058]]
----------------time= 9 -------------------------
M= 1 avg log_likelihood of test [[-1.00608294]]
----------------time

The result above is perform pretty similar to"average likelihood of M= [-1.0016]"
in MLE of same data set  

So, below I try to find another hyperparameter a to achieve better performance

finding best a for M=1

In [12]:
def cross_validation_MAP(X,Y,cut_size,Max_order,a):

  X_sets=N_set(X,cut_size)
  Y_sets=N_set(Y,cut_size)
  times=len(X_sets)

  count=[np.float64(0)]*Max_order
  for t in range(times):
    #print("----------time=",t,"-------------------------")
    X_train=pd.DataFrame()
    Y_train=pd.DataFrame()
    X_test=X_sets[t].copy()
    Y_test=Y_sets[t].copy()
    for i in range(times):
      if i==t :
        continue
      else:
        X_train=X_train.append(X_sets[i])
        Y_train=Y_train.append(Y_sets[i])
    tmp=Validation_MAP_poly(X_train, X_test, Y_train, Y_test,a, Max_order,if_print=False)
    for i in range(Max_order):
      count[i]+=tmp[i]
  #print("############################################")
  for i in range(Max_order):
    count[i]/=times
    print("average likelihood of M=",i+1,": ",count[i])
  return count

In [13]:
epoch=10
max_order=1
a=0.0001
cut_size=100

step=np.geomspace(0.1,1,epoch)
likely_max=[np.float64(-1000)]*max_order

a_max=a
print(step)
for time_epoch in range(epoch):
  print("##---------------epoch:",time_epoch,"------------------##")
  a=step[time_epoch]
  cur_likely=cross_validation_MAP(X,Y,cut_size,max_order,a)
  for i in range(max_order):
    if cur_likely[i] > likely_max[i]:
      print("!!----------------------------------------------------!!")
      print("Update New a: ",a,"New likelihood ",cur_likely[i])
      likely_max[i]=cur_likely[i]
      a_max=a
      print("!!----------------------------------------------------!!")
    else:
      print("a: ",a,"likelihood: ",cur_likely[i])
print("Final a: ",a_max,"Final likelihood: ",likely_max)

[0.1        0.12915497 0.16681005 0.21544347 0.27825594 0.35938137
 0.46415888 0.59948425 0.77426368 1.        ]
##---------------epoch: 0 ------------------##
average likelihood of M= 1 :  [[-0.99935537]]
!!----------------------------------------------------!!
Update New a:  0.1 New likelihood  [[-0.99935537]]
!!----------------------------------------------------!!
##---------------epoch: 1 ------------------##
average likelihood of M= 1 :  [[-0.99932233]]
!!----------------------------------------------------!!
Update New a:  0.1291549665014884 New likelihood  [[-0.99932233]]
!!----------------------------------------------------!!
##---------------epoch: 2 ------------------##
average likelihood of M= 1 :  [[-0.9992843]]
!!----------------------------------------------------!!
Update New a:  0.16681005372000587 New likelihood  [[-0.9992843]]
!!----------------------------------------------------!!
##---------------epoch: 3 ------------------##
average likelihood of M= 1 :  [[-0.99

So,the a in the interval(1~0.1) are perform well compared to other interval,  
In this coarse trial we found a=0.5995 is a good choice for hyperparameter a.  
Or we can get more precise result by applying other optimizing algorithm like GD.

**-------------------------Conclusion--------------------------**

The data is never enough so MLE never find the "real" underlying parameter,but the assumption based on data.  
But, MAP providing us an approach or a chance to reach the "real" underlying parameter beyond the scope of data  
By taking an appropriate prior, we can have a regulizing approach acquiring more "reasonable" prediction
  
Above results verify that we can adjust our prior(alpha in this case) to get higher likelihood than MLE approch.  
same data in MLE:average cross validation likelihood of M= 1 :  [-1.00224412]  
but in MAP: average cross validation likelihood of M= 1 :  [[-0.99909072]]  

So, with goood prior we can obtain better prediction